In [1]:
import math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime,timedelta
from copy import copy
import seaborn as sns

from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split

import xgboost as xgb

In [2]:
#define the constants used in the file
file_suffix = '.csv'
train_path = '../data/dataSets/training/'
test_path = '../data/dataSets/testing_phase1/'

In [3]:
#read train data
train = pd.read_csv('training_20min_avg_travel_time.csv') 
train.head()

,intersection_id,tollgate_id,time_window,averagetravltime,lag1,lag2,lag3,lag4,lag5,lag6,...,wind_direction,wind_speed,temperature,rel_humidity,precipitation,link_count,route_length,in_link,out_link,min_width
0,B,3,"[2016-07-19 00:00:00,2016-07-19 00:20:00)",70.85,119.257934,119.256358,119.256739,119.256667,119.255553,119.257916,...,239.0,1.9,31.8,64.0,0.0,5,477,2,1,3
1,A,2,"[2016-07-19 00:20:00,2016-07-19 00:40:00)",58.05,70.850000,119.256358,119.256739,119.256667,119.255553,119.257916,...,239.0,1.9,31.8,64.0,0.0,6,384,0,1,3
2,B,1,"[2016-07-19 00:20:00,2016-07-19 00:40:00)",79.76,58.050000,70.850000,119.256739,119.256667,119.255553,119.257916,...,239.0,1.9,31.8,64.0,0.0,9,821,2,1,6
3,B,3,"[2016-07-19 00:20:00,2016-07-19 00:40:00)",148.79,79.760000,58.050000,70.850000,119.256667,119.255553,119.257916,...,239.0,1.9,31.8,64.0,0.0,5,477,2,1,3
4,B,1,"[2016-07-19 00:40:00,2016-07-19 01:00:00)",137.98,148.790000,79.760000,58.050000,70.850000,119.255553,119.257916,...,239.0,1.9,31.8,64.0,0.0,9,821,2,1,6


In [4]:
cat_columns = ["intersection_id","tollgate_id"]
df_train = pd.get_dummies(train, prefix_sep="__",
                              columns=cat_columns)

In [5]:
print(df_train.shape)

(25592, 35)


In [6]:
print(df_train.shape)

(25592, 35)


In [7]:
df_train.head()

,time_window,averagetravltime,lag1,lag2,lag3,lag4,lag5,lag6,lag7,date,...,route_length,in_link,out_link,min_width,intersection_id__A,intersection_id__B,intersection_id__C,tollgate_id__1,tollgate_id__2,tollgate_id__3
0,"[2016-07-19 00:00:00,2016-07-19 00:20:00)",70.85,119.257934,119.256358,119.256739,119.256667,119.255553,119.257916,119.255205,2016-07-19,...,477,2,1,3,0,1,0,0,0,1
1,"[2016-07-19 00:20:00,2016-07-19 00:40:00)",58.05,70.850000,119.256358,119.256739,119.256667,119.255553,119.257916,119.255205,2016-07-19,...,384,0,1,3,1,0,0,0,1,0
2,"[2016-07-19 00:20:00,2016-07-19 00:40:00)",79.76,58.050000,70.850000,119.256739,119.256667,119.255553,119.257916,119.255205,2016-07-19,...,821,2,1,6,0,1,0,1,0,0
3,"[2016-07-19 00:20:00,2016-07-19 00:40:00)",148.79,79.760000,58.050000,70.850000,119.256667,119.255553,119.257916,119.255205,2016-07-19,...,477,2,1,3,0,1,0,0,0,1
4,"[2016-07-19 00:40:00,2016-07-19 01:00:00)",137.98,148.790000,79.760000,58.050000,70.850000,119.255553,119.257916,119.255205,2016-07-19,...,821,2,1,6,0,1,0,1,0,0


In [8]:
train_label = df_train['averagetravltime']

In [9]:
df_train.drop(['averagetravltime','time_window','date','month','day','hour','minute'], axis=1, inplace=True)
train_data = df_train

In [10]:
X_train, X_test, y_train, y_test = train_test_split(train_data, train_label, test_size=0.2, random_state=123)

In [11]:
data_dmatrix = xgb.DMatrix(data=train_data ,label=train_label)

xg_reg = xgb.XGBRegressor(objective ='reg:linear', eta = 0.02, min_child_weight = 8,
                subsample = 0.9, colsample_bytree = 0.8, max_depth = 8,
                seed=1, silent=1)

D:\anaconda\conda3\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
D:\anaconda\conda3\lib\site-packages\xgboost\core.py:588: FutureWarning: Series.base is deprecated and will be removed in a future version
  data.base is not None and isinstance(data, np.ndarray) \


In [12]:
xg_reg.fit(X_train,y_train)

preds = xg_reg.predict(X_test)

In [13]:
rmse = np.sqrt(mean_squared_error(y_test, preds))
print("RMSE: %f" % (rmse))

RMSE: 38.694482


In [14]:
def mape(label, pred):

    return np.mean(np.abs(label - pred).astype(np.float64) / label)

In [15]:
test_err = mape(y_test, preds)
print ("MAPE: %f" % (test_err))

MAPE: 0.242141
